In [48]:
%load_ext autoreload
%autoreload 2

import os
import json
import genesis_cloud.utils as u
from tqdm import tqdm
from step.step import *
from llama_cpp import Llama, LlamaGrammar
import time
import random
instance_url = '147.189.192.78:8080'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Grammar based 

In [76]:
table_1 = [
['MYCOPHENOLATE MOFETIL', '1000 MG PO BID', 'BID', ''],
  ['OXYBUTYNIN CHLORIDE XL', '10 MG PO QD', 'QD', ''],
  ['INSULIN GLARGINE', '20 UNITS SC QAM', 'QAM', ''],
  ['FUROSEMIDE', 'PO QD', 'QD', ''],
  ['CLOPIDOGREL', '75 MG PO QD', 'QD', ''],
  ['PRAVASTIN', '40 MG PO QHS', 'QHS', ''],
  ['PREDNISONE', '5 MG PO QD', 'QD', ''],
  ['CYCLOSPORINE (SANDIMMUNE)', '75 MG PO BID', 'BID', ''],
  ['METOPROLOL SUCCINATE EXTENDED RELEASE', '50 MG PO QD', 'QD', ''],
  ['FENOFIBRATE (TRICOR)', '48 MG PO QD', 'QD', ''],
  ['ENTERIC COATED ASA', '325 MG PO DAILY', 'DAILY', ''],
  ['PLAVIX (CLOPIDOGREL)', '75 MG PO DAILY', 'DAILY', ''],
  ['CYCLOSPORINE (SANDIMMUNE)', '75 MG PO BID', 'BID', ''],
  ['LASIX (FUROSEMIDE)', '40 MG PO DAILY', 'DAILY', ''],
  ['INSULIN GLARGINE', '20 UNITS SC DAILY', 'DAILY', ''],
  ['CELLCEPT (MYCOPHENOLATE MOFETIL)', '1, 000 MG PO BID', 'BID', ''],
  ['TNG 0.4 MG (NITROGLYCERIN 1/150 (0.4 MG))','1 TAB SL Q5MIN X 3 doses PRN Chest Pain','PRN',''],
  ['PRAVACHOL (PRAVASTIN)', '40 MG PO BEDTIME', 'BEDTIME', ''],
  ['PREDNISOLONE SODIUM PHOSPHATE', '5MG/5ML 5 MG PO DAILY', 'DAILY', '']]
table_2 = [
 ['mycophenolate mofetil', '1000 mg', 'po', 'bid', '19'],
 ['oxybutynin chloride xl', '10 mg', 'po', 'qd', '20'],
 ['insulin glargine', '20 units', 'sc', 'qam', '21'],
 ['furosemide', '', 'po', 'qd', '22'],
 ['clopidogrel', '75 mg', 'po', 'qd', '23'],
 ['pravastatin', '40 mg', 'po', 'qhs', '24'],
 ['prednisone', '5 mg', 'po', 'qd', '25'],
 ['cyclosporine ( sandimmune )', '75 mg', 'po', 'bid', '26'],
 ['metoprolol succinate extended release', '50 mg', 'po', 'qd', '27'],
 ['fenofibrate ( tricor )', '48 mg', 'po', 'qd', '28'],
 ['enteric coated asa', '325 mg', 'po', 'daily', '30'],
 ['plavix ( clopidogrel )', '75 mg', 'po', 'daily', '31'],
 ['cyclosporine ( sandimmune )', '75 mg', 'po', 'bid', '33'],
 ['tricor', '', 'po', '', '37'],
 ['cyclosporine', '', '', '', '38'],
 ['fenofibrate , micronized', '', '', '', '39'],
 ['micronized', '', '', '', '39'],
 ['home meds', '', '', '', '40'],
 ['pravastatin sodium', '', '', '', '42']]

In [77]:
table_2 = [row[:4] for row in table_2]

In [78]:
prompt = """
Two extracted rows are considered to contain the same information if this expression holds true:
(The first column of both rows contain the same information) 
AND (
    (The second column of both rows contain the same information) 
    OR
    (The third column of both rows contain the same information)
    OR
    (The fourth column of both rows contain the same information)
)
"""

In [79]:
grammar = """
root ::= \"\n
"""
for i, j in zip(table_1, table_2):
    grammar += "row 1: " + ";".join(i) + "\n"
    grammar += "row 2: " + ";".join(j) + "\n"
    grammar += "Contains same information: \" boolean"
    grammar += "\"\\n---\n"

grammar += "\"\nboolean ::= \"true\" | \"false\""
print(grammar)


root ::= "

row 1: MYCOPHENOLATE MOFETIL;1000 MG PO BID;BID;
row 2: mycophenolate mofetil;1000 mg;po;bid
Contains same information: " boolean"\n---
row 1: OXYBUTYNIN CHLORIDE XL;10 MG PO QD;QD;
row 2: oxybutynin chloride xl;10 mg;po;qd
Contains same information: " boolean"\n---
row 1: INSULIN GLARGINE;20 UNITS SC QAM;QAM;
row 2: insulin glargine;20 units;sc;qam
Contains same information: " boolean"\n---
row 1: FUROSEMIDE;PO QD;QD;
row 2: furosemide;;po;qd
Contains same information: " boolean"\n---
row 1: CLOPIDOGREL;75 MG PO QD;QD;
row 2: clopidogrel;75 mg;po;qd
Contains same information: " boolean"\n---
row 1: PRAVASTIN;40 MG PO QHS;QHS;
row 2: pravastatin;40 mg;po;qhs
Contains same information: " boolean"\n---
row 1: PREDNISONE;5 MG PO QD;QD;
row 2: prednisone;5 mg;po;qd
Contains same information: " boolean"\n---
row 1: CYCLOSPORINE (SANDIMMUNE);75 MG PO BID;BID;
row 2: cyclosporine ( sandimmune );75 mg;po;bid
Contains same information: " boolean"\n---
row 1: METOPROLOL SUCCINATE EX

In [6]:
%%time
out = u.call_model(instance_url, prompt, grammar=grammar)
print(out)



row 1: MYCOPHENOLATE MOFETIL;1000 MG PO BID;BID;
row 2: mycophenolate mofetil;1000 mg;po;bid
Contains same information: true
---
row 1: OXYBUTYNIN CHLORIDE XL;10 MG PO QD;QD;
row 2: oxybutynin chloride xl;10 mg;po;qd
Contains same information: true
---
row 1: INSULIN GLARGINE;20 UNITS SC QAM;QAM;
row 2: insulin glargine;20 units;sc;qam
Contains same information: false
---
row 1: FUROSEMIDE;PO QD;QD;
row 2: furosemide;nm;po;qd
Contains same information: false
---
row 1: CLOPIDOGREL;75 MG PO QD;QD;
row 2: clopidogrel;75 mg;po;qd
Contains same information: true
---
row 1: PRAVASTIN;40 MG PO QHS;QHS;
row 2: pravastatin;40 mg;po;qhs
Contains same information: true
---
row 1: PREDNISONE;5 MG PO QD;QD;
row 2: prednisone;5 mg;po;qd
Contains same information: true
---
row 1: CYCLOSPORINE (SANDIMMUNE);75 MG PO BID;BID;
row 2: cyclosporine ( sandimmune );75 mg;po;bid
Contains same information: true
---
row 1: METOPROLOL SUCCINATE EXTENDED RELEASE;50 MG PO QD;QD;
row 2: metoprolol succinate exte

### Prompt based

In [80]:
%%time

output2 = []
for _ in range(3):
    prompt = """
If the 50% or more of the information in the two rows is the same, consider them to contain the same information.
Notice that order does not matter. 

"""
    output_tmp = []
    joint = list(enumerate(zip(table_1, table_2)))
    random.shuffle(joint)
    for row in joint:
        prompt += "row 1: " + " ".join(row[1][0]) + "\n"
        prompt += "row 2: " + " ".join(row[1][1]) + "\n"
        prompt += "Contains same information: "
        out = u.call_model(instance_url,  prompt, temperature=0.8, grammar="root ::= \"true\n\" | \"false\n\"")
        if "true" in out:
            out = "true"
        else:
            out = "false"
        output_tmp.append([row[0], out])
        prompt += out + '\n---\n'
    output_tmp = sorted(output_tmp, key=lambda x: x[0])
    output_tmp = [x[1] for x in output_tmp]
    output2.append(output_tmp)

CPU times: user 229 ms, sys: 48.2 ms, total: 277 ms
Wall time: 24.9 s


In [81]:
output = output2

In [82]:
### convert true in +1 and false in -1
output = [[1 if x == "true" else -1 for x in y] for y in output]

### sum column wise
output = [sum(x) for x in zip(*output)]

In [83]:
output = [(x, True) if x > 0 else (x, False) for x in output]

In [84]:
rows_1 = [" ".join(row) for row in table_1]
rows_2 = [" ".join(row) for row in table_2]
list(zip(rows_1, rows_2, output))

[('MYCOPHENOLATE MOFETIL 1000 MG PO BID BID ',
  'mycophenolate mofetil 1000 mg po bid',
  (3, True)),
 ('OXYBUTYNIN CHLORIDE XL 10 MG PO QD QD ',
  'oxybutynin chloride xl 10 mg po qd',
  (3, True)),
 ('INSULIN GLARGINE 20 UNITS SC QAM QAM ',
  'insulin glargine 20 units sc qam',
  (3, True)),
 ('FUROSEMIDE PO QD QD ', 'furosemide  po qd', (3, True)),
 ('CLOPIDOGREL 75 MG PO QD QD ', 'clopidogrel 75 mg po qd', (3, True)),
 ('PRAVASTIN 40 MG PO QHS QHS ', 'pravastatin 40 mg po qhs', (1, True)),
 ('PREDNISONE 5 MG PO QD QD ', 'prednisone 5 mg po qd', (3, True)),
 ('CYCLOSPORINE (SANDIMMUNE) 75 MG PO BID BID ',
  'cyclosporine ( sandimmune ) 75 mg po bid',
  (3, True)),
 ('METOPROLOL SUCCINATE EXTENDED RELEASE 50 MG PO QD QD ',
  'metoprolol succinate extended release 50 mg po qd',
  (3, True)),
 ('FENOFIBRATE (TRICOR) 48 MG PO QD QD ',
  'fenofibrate ( tricor ) 48 mg po qd',
  (3, True)),
 ('ENTERIC COATED ASA 325 MG PO DAILY DAILY ',
  'enteric coated asa 325 mg po daily',
  (3, True))